<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/makePlan_from_gsm8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install peft
!pip install datasets
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.5 MB/s eta 0:00:00


In [2]:
from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset

In [4]:
from google.colab import userdata

from huggingface_hub import login
login(token=userdata.get('HUG_TOKEN'))

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
### 4bit quant ###

# model_path = "meta-llama/Llama-3.1-8B-Instruct"

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Quantization configuration
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# # Load Tokenizer and Model
# tokenizer = AutoTokenizer.from_pretrained(
#     model_path,
#     model_max_length=512,
#     padding_side="left",
#     add_eos_token=True,
# )
# tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     quantization_config=bnb_config,
#     # device_map="auto",
#     # trust_remote_code=True,
#     # use_auth_token=True,
# )

# model.eval()

In [5]:
### Load Model Normaly ###

model_path = "meta-llama/Llama-3.1-8B-Instruct"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

# Load Model without quantization
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,  # GPU 메모리 절약을 위해 float16 사용 (필요시 float32로 변경 가능)
    device_map="auto",  # 모델을 자동으로 GPU로 로드
)

model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (n

In [7]:
dataset = load_dataset("openai/gsm8k", 'main')
dataset

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1319
    })
})

In [12]:
from tqdm import tqdm
import csv

csv_file = "generated_plan-gsm8k.csv"

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

start_row = 0
max_snippet_length = 1024
sliced_dataset = dataset['train'].select(range(0, 1000))
last_prompt = ""


with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["question", "answer", "plan"])

    for row in tqdm(sliced_dataset, desc="Generating plans", unit="sample"):
        problem = row['question']
        answer = row['answer']

        prompt = f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

<|eot_id|>
<|start_header_id|>user<|end_header_id|>
You are a Python programming assistant. Your task is to create a concise step-by-step plan for writing Python code to solve the given problem. Use the provided problem and its explanation of the answer to guide your plan. Focus on minimal and essential steps. Do not generate Python code.

Problem: {problem}

Answer Explanation: {answer}

Provide a concise plan:
1. Define the required variables and their initial values.
2. Describe the step-by-step logic to calculate the answer.
<|start_header_id|>assistant<|end_header_id|>
"""

        inputs = tokenizer(prompt, return_tensors="pt", max_length=512, truncation=True).to(device)

        # 입력 토큰의 길이 계산
        input_length = inputs['input_ids'].shape[1]

        # 입력 프롬프트 토큰 수에 128을 더한 max_length 설정
        max_length = input_length + 256

        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_length=max_length,
                eos_token_id=terminators,
                pad_token_id=tokenizer.eos_token_id,
                temperature=0.2
            )

            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

            generated_plan = ""

            keyword = "<|start_header_id|>assistant<|end_header_id|>"
            if keyword in generated_text:
                generated_plan = generated_text.split(keyword)[1].strip()

            keyword = "<|eot_id|>"
            if keyword in generated_plan:
                generated_plan = generated_plan.split(keyword)[0].strip()

            if not generated_plan:
                generated_plan = "none"
                # continue

            # print(generated_plan)
            # break
            writer.writerow([problem, answer, generated_plan])




Generating plans: 100%|██████████| 1000/1000 [3:46:58<00:00, 13.62s/sample]


In [13]:
# !cp generated_plan0-3000.csv /content/drive/MyDrive/


In [14]:
import pandas as pd

df = pd.read_csv(csv_file)
# print(df['question'][0])
# print(df['plan'][0])

In [15]:
print(df.columns)
print(df.shape)

Index(['question', 'answer', 'plan'], dtype='object')
(1000, 3)


In [16]:
from datasets import Dataset
from google.colab import userdata

plan_dataset = Dataset.from_pandas(df)
plan_dataset.push_to_hub("donghuna/gsm8k_with_plan", token=userdata.get('HUG_TOKEN'))

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/donghuna/gsm8k_with_plan/commit/1fe4a4f160b17fb7531f7a52b315ed2308a5373f', commit_message='Upload dataset', commit_description='', oid='1fe4a4f160b17fb7531f7a52b315ed2308a5373f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/donghuna/gsm8k_with_plan', endpoint='https://huggingface.co', repo_type='dataset', repo_id='donghuna/gsm8k_with_plan'), pr_revision=None, pr_num=None)